In [1]:
	using CairoMakie, ColorSchemes
	import SpecialFunctions: erfc, erfcinv


	include("D:\\MAIN\\MODELS\\AquaPore_Toolkit\\src\\Cst.jl")

Main.cst

In [2]:
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
		#		FUNCTION : ∂θ∂Ψ
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
			function ∂θ∂ΨMODEL(Ψ₁, θs, θr, Ψm, σ, θsMacMat, ΨmMac, σmac)

			Ψmod_Mat = exp(log(Ψm) - σ^2)

			Ψmod_Mac = exp(log(ΨmMac) - σmac^2)

				# Ψ₁ = max(eps(10.0), Ψ₁)
			
					# If Ψ₁ is positive than ∂θ∂Ψ_Mat should be positive
					∂θ∂Ψ_Mat = (θsMacMat - θr) * exp( -((log(Ψ₁ / Ψm)) ^ 2.0) / (2.0 * σ^2.0)) / (Ψ₁ * σ * √(π * 2.0))

					∂θ∂Ψ_Mat_Mod = (θsMacMat - θr) * exp( -((log(Ψmod_Mat / Ψm)) ^ 2.0) / (2.0 * σ^2.0)) / (Ψmod_Mat * σ * √(π * 2.0))

					∂θ∂Ψ_Mac = (θs - θsMacMat) * exp( -((log(Ψ₁ / ΨmMac)) ^ 2.0) / (2.0 * σmac^2.0)) / (Ψ₁ * σmac * √(π * 2.0))

					∂θ∂Ψ_Mac_Mod = (θs - θsMacMat) * exp( -((log(Ψmod_Mac / ΨmMac)) ^ 2.0) / (2.0 * σmac^2.0)) / (Ψmod_Mac * σmac * √(π * 2.0))

				
					return (∂θ∂Ψ_Mat / ∂θ∂Ψ_Mat_Mod + ∂θ∂Ψ_Mac / ∂θ∂Ψ_Mac_Mod) * 0.5

			end # function ∂θ∂Ψ
		#-------------------------------------------------------------------

∂θ∂ΨMODEL (generic function with 1 method)

In [3]:
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
		#		FUNCTION : Ψ_2_θ
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
			function Ψ_2_θDualMODEL(Ψ₁, θs, θr, Ψm, σ, θsMacMat, ΨmMac, σmac)

				θ_Mat = 0.5 * (θsMacMat - θr) * erfc((log( Ψ₁ / Ψm)) / (σ * √2.0)) + θr

				θ_Mac = 0.5 * (θs - θsMacMat) * erfc((log(Ψ₁ / ΨmMac)) / (σmac * √2.0))
			return θ_Mac + θ_Mat
			end # function Ψ_2_θ
		#-----------------------------------------------------------------

Ψ_2_θDualMODEL (generic function with 1 method)

In [4]:
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
		#		FUNCTION : KUNSAT_θΨSe
		# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
			function Ψ_2_KUNSAT_MODEL(Ψ₁, θs, θr, Ψm, σ, θsMacMat, ΨmMac, σmac, Ks)

				θ = Ψ_2_θDualMODEL(Ψ₁, θs, θr, Ψm, σ, θsMacMat, ΨmMac, σmac)

				Se = (θ - θr) / (θs - θr)

				KsMat = Ks * min(max((θsMacMat - θr) / (θs - θr), 0.0), 1.0)			
				Kunsat_Mat =  KsMat * √Se * (0.5 * erfc(((log(Ψ₁ / Ψm)) / σ + σ) / √2.0)) ^ 2.0

				KsMac = max(Ks - KsMat, 0.0)
				Kunsat_Mac =  KsMac * √Se * (0.5 * erfc(((log(Ψ₁ / ΨmMac)) / σmac + σmac) / √2.0)) ^ 2.0

			return Kunsat_Mat + Kunsat_Mac
			end # function KUNSAT_θΨSe
		#-------------------------------------------------------------------

Ψ_2_KUNSAT_MODEL (generic function with 1 method)

In [5]:
Pσ = 4.0
ΨmacMat = 100.0
ΨmMac = √ΨmacMat
σmac = log(ΨmacMat) / (2.0 * 3)
θs = 0.5
θr = 0
σ = 1.5
Ψm = (√ΨmacMat + ΨmacMat) * 0.5 * exp(σ * Pσ)
θsMacMat = θs * 0.75
Ks = 0.008


0.008

In [18]:
Ψ = 10.0.^(collect(0:0.001:6))
N = length(Ψ)
∂θ∂Ψ_1 = fill(0.0::Float64 , N)
∂θ∂Ψ_2 = fill(0.0::Float64 , N)

Ψ_2_θDual_1= fill(0.0::Float64 , N)
Ψ_2_θDual_2= fill(0.0::Float64 , N)

Ψ_2_KUNSAT_1 = fill(0.0::Float64 , N)
Ψ_2_KUNSAT_2 = fill(0.0::Float64 , N)  

for iΨ=1:N
	∂θ∂Ψ_1[iΨ] = ∂θ∂ΨMODEL(Ψ[iΨ], θs, θr, ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac)
	∂θ∂Ψ_2[iΨ] = ∂θ∂ΨMODEL(Ψ[iΨ], θs, θr, √ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac)

	Ψ_2_θDual_1[iΨ] = Ψ_2_θDualMODEL(Ψ[iΨ], θs, θr, ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac)
	Ψ_2_θDual_2[iΨ] = Ψ_2_θDualMODEL(Ψ[iΨ], θs, θr, √ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac)

	Ψ_2_KUNSAT_1[iΨ] = Ψ_2_KUNSAT_MODEL(Ψ[iΨ], θs, θr, ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac, Ks)

	Ψ_2_KUNSAT_2[iΨ] = Ψ_2_KUNSAT_MODEL(Ψ[iΨ], θs, θr, √ΨmacMat * exp(σ * Pσ), σ, θsMacMat, ΨmMac, σmac, Ks)
	
end

In [23]:

Fig = Figure(resolution = (2500, 1000),  font="Sans", fontsize=50)
Linewidth = 4

Axis1 = Axis(Fig[1,1], title="∂θ∂Ψ", xlabel="log R [mm]", ylabel="∂θ∂Ψ", xscale=Makie.log)
	lines!(Axis1, Ψ , ∂θ∂Ψ_1,  color=:green, linewidth=Linewidth)
	lines!(Axis1, Ψ , ∂θ∂Ψ_2, color=:blue, linewidth=Linewidth)
	Axis1.xticks = (log.(cst.Mm_2_kPa .* Ψ), string.( floor.(cst.Mm_2_kPa .*  log.(Ψ), digits=1)))
	# xlims!(Axis1, log(cst.Mm_2_kPa * Ψ_θΨobs_Min), log1p.(cst.Mm_2_kPa * Ψ_θΨobs_Max ))
	# ylims!( Axis1, 0.0, max(hydro.Φ[iZ], maximum(θ_θΨobs[iZ,1:N_θΨobs[iZ]])) )


Axis2 = Axis(Fig[2,1], title="θ(Ψ)", xlabel="Ψ [kPa]", ylabel="θ(Ψ)", xscale=Makie.log)
	lines!(Axis2, Ψ , Ψ_2_θDual_1, color=:green, linewidth=Linewidth)
	lines!(Axis2, Ψ , Ψ_2_θDual_2, color=:blue, linewidth=Linewidth)
	Axis2.xticks = (log.(cst.Mm_2_kPa .* Ψ), string.( floor.(cst.Mm_2_kPa .*  log.(Ψ), digits=1)))

Axis3 = Axis(Fig[3,1], title="K(Ψ)", xlabel="Ψ [kPa]", ylabel="K(Ψ)", xscale=Makie.log)
lines!(Axis3, Ψ , Ψ_2_KUNSAT_1, color=:green, linewidth=Linewidth)
lines!(Axis3, Ψ , Ψ_2_KUNSAT_2, color=:blue, linewidth=Linewidth)
Axis3.xticks = (log.(cst.Mm_2_kPa .* Ψ), string.( floor.(cst.Mm_2_kPa .*  log.(Ψ), digits=1)))

trim!(Fig.layout)
Leg.orientation = :horizontal
Leg.tellheight = true

display(Fig)